In [3]:
# -*- coding: utf-8 -*-
import types
import csv
import re
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
# NLP module
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer #名称変更
"線形SVMを利用するためにLinearSVMクラスをインポート"
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
"学習データと評価データを分割するための関数をインポート"
from sklearn.cross_validation import train_test_split

import MeCab
from sklearn.linear_model import LogisticRegression
tagger = MeCab.Tagger("-Owakati")

from sklearn.metrics import confusion_matrix 

output1 = []
output2 = []

f1 = open('/Users/yukihiro-su/Desktop/genso.csv') # 旅行のカテゴリのタイトル
data1 = f1.read()  # ファイル終端まで全て読んだデータを返す
f1.close()

f2 = open('/Users/yukihiro-su/Desktop/mazime.csv') # 政治のカテゴリのタイトル
data2 = f2.read()  # ファイル終端まで全て読んだデータを返す
f2.close()

lines1 = data1.split(',') 
lines2 = data2.split(',') 

for line in lines1:
    #print (line)
    line = re.sub('[!-/:-@[-`{-~]', "", line) #半角記号
    line = re.sub('u"[︰-＠]"', "", line) #全角記号
    line = re.sub('\n',"",line)
    line = tagger.parse(line)
    output1.append(line)

for line in lines2:
    #print (line)
    line = re.sub('[!-/:-@[-`{-~]', "", line) #半角記号
    line = re.sub('u"[︰-＠]"', "", line) #全角記号
    line = re.sub('u"【】「」"', "", line) #全角記号
    line = re.sub('\n',"",line)
    line = tagger.parse(line)
    output2.append(line)
    


a1= np.array(output1)
print(a1)
a1.reshape(a1.shape[0],1)
a2= np.array(output2)
print(a2)
a2.reshape(a2.shape[0],1)
a = np.r_[a1,a2]


# CountVectorizer
#count = CountVectorizer()
#bag = count.fit_transform(a)
#print(count.vocabulary_)
#print(bag.toarray())

# TF-IDF
tfidf = TfidfVectorizer(use_idf=True, norm='l2', smooth_idf=True)
X = tfidf.fit_transform(a)
#np.set_printoptions(precsion=2) #新しいバージョンから使用しない
#print(vecs.toarray())

b1 = [1]*(a1.shape[0])
b1 = np.array(b1)
b1.reshape(a1.shape[0],1)
b2 = [2]*(a2.shape[0])
b2 = np.array(b2)
b2.reshape(a2.shape[0],1)
#result = np.dstack((a,b)) #リストで返したい時
y = np.r_[b1,b2]

#print(x.shape[0])
#print(x.shape[1])
#print(y.shape[0])


"特徴量と正解ラベルを学習データと評価データへ分割(一応、二割を評価用にする。)"
"data_train,data_test,label_train,label_test = train_test_split(data,mnist.target,test_size=0.2)"
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=0)

"チューニングを行なって各種パラメータを調整する"
parameters = [
       {'C': [1, 10, 100, 1000]}
]

clf_svm = GridSearchCV(SVC(), parameters, cv=5, scoring='accuracy', n_jobs=-1)
clf_svm.fit(X_train, y_train)
lr2 = GridSearchCV(LogisticRegression(penalty='l2',multi_class='multinomial',solver='saga'), param_grid=parameters, cv=5, scoring='accuracy', n_jobs=-1)
lr2.fit(X_train, y_train)
result_svm = clf_svm.predict(X_test)
result_lr2 = lr2.predict(X_test)

"accuracyを求めて、格納する。"
accuracy1 = accuracy_score(y_test, result_svm)
accuracy2 = accuracy_score(y_test, result_lr2)
print(accuracy1)
print(accuracy2)
cmat = confusion_matrix(y_test, result_lr2)
print(cmat)


output = []
# CountVectorizer
count = CountVectorizer()

#str = "毎日残業疲れたから、もう幻想郷に旅行に行きたい"
result = tagger.parse("毎日サービス残業疲れたから、ハワイ、ドバイに行きたい")
output.append(result)
fin = tfidf.transform(output)
print(fin)
print(lr2.predict(fin))



['ドイツ の 旅 ２ 「 世界 遺産 \u3000 ハイデルベルク 城 」 \n' 'タイ の 危険 地帯 \n'
 '【 無計画 】 もう やん の 自転車 １ 週間 １ 人 旅 ～ ４ \n' ...,
 '臨時 急行 き た ぐに \u3000 京都 駅 発車 \n' '京王 線 ・ 地上 へ ！ \n'
 '錦江湾 サマー ナイト 大 花火 大会 2014 鹿児島 市 ダイジェスト \n']
['AW ニュース Weekly 『 昭和 天皇 実録 』 公表 \u3000 100 \n'
 '侍 ＪＰ さん と お話 です 。 vol 1 \n'
 '【 政見 放送 】 2010 参院 選 東京 都 選挙 区 \u3000 松本 みのる \n' ...,
 '【 新 唐人 】 演壇 から 下ろさ れ た 有名 教授 と 裸 の 京劇 \n'
 '日本 が 韓国 を 近代 国家 に し て しまっ た こと を 謝罪 しよ う パレード ⑤ \n'
 '字幕 【 テキサス 親父 】 \u3000 また も 笑わせる シー・シェパード ！ 危機 ？ \n']


/Users/yukihiro-su/.pyenv/versions/anaconda3-4.0.0/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/yukihiro-su/.pyenv/versions/anaconda3-4.0.0/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/yukihiro-su/.pyenv/versions/anaconda3-4.0.0/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/yukihiro-su/.pyenv/versions/anaconda3-4.0.0/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/y

0.911122144985
0.941410129096
[[955  42]
 [ 76 941]]
  (0, 13785)	0.321620610544
  (0, 11658)	0.425596245202
  (0, 5699)	0.372989146524
  (0, 5525)	0.446077558363
  (0, 4941)	0.460609281312
  (0, 3453)	0.337976110415
  (0, 3159)	0.225683774785
[1]
